In [1]:
import camelot
data = camelot.read_pdf('bulletin.pdf', pages='140-147,161-200', flavor='lattice')

In [2]:
def find_cdcs(s):
    s = ''.join(i for i in s if i!='\n')
    s = s.split()
    ans = list()
    for i in range(1,len(s)):
        if s[i-1].isupper() and s[i][0]=='F' and s[i][1:].isdigit() :
            ans.append(s[i-1] + '_' + s[i])
    return ans
branchcode = {
    "Biological":'B1',
    "Chemistry":'B2',
    "Economics":'B3',
    "Mathematics":'B4',
    "Physics":'B5',
    "Chemical":'A1',
    "Computer":'A7',
    "Electrical":'A3',
    "Communication":'AA',
    "Instrumentation":'A2',
    "Mechanical":'A4'        
}

cdcs = dict()
def make_dict(df):
    bc = ''
    for i in branchcode.keys():
        if (df[0][0].find(i, 50) >= 0):
            bc = bc + branchcode[i]
    
    if (bc=='' or bc[-2]=='B' or bc in cdcs.keys()):
        return
    
    sem = dict()
    roman = {'II':2, 'III':3, 'IV':4, 'V':5}
    for i in df.index:
        if df[0][i] in roman.keys():
            if len(bc)==2:
                sem[2*roman[df[0][i]]-1] = find_cdcs(df[1][i])
                sem[2*roman[df[0][i]]] = find_cdcs(df[3][i])
            else:
                sem[2*roman[df[0][i]]-1] = find_cdcs(df[1][i+1])
                sem[2*roman[df[0][i]]] = find_cdcs(df[3][i+1])
    cdcs[bc] = sem

In [3]:
for i in range(len(data)):
    make_dict(data[i].df)

In [4]:
cdcs

{'A1': {3: ['MATH_F211',
   'CHE_F211',
   'CHE_F214',
   'CHE_F213',
   'CHE_F212',
   'BITS_F225'],
  4: ['ECON_F211',
   'MGTS_F211',
   'CHE_F241',
   'CHE_F242',
   'CHE_F243',
   'CHE_F244'],
  5: ['CHE_F312', 'CHE_F313', 'CHE_F311', 'CHE_F314'],
  6: ['CHE_F341', 'CHE_F342', 'CHE_F343'],
  7: [],
  8: ['BITS_F412']},
 'A7': {3: ['MATH_F211', 'CS_F214', 'CS_F222', 'CS_F213', 'CS_F215'],
  4: ['ECON_F211', 'MGTS_F211', 'CS_F211', 'CS_F241', 'CS_F212', 'BITS_F225'],
  5: ['CS_F351', 'CS_F372', 'CS_F301', 'CS_F342'],
  6: ['CS_F363', 'CS_F364', 'CS_F303'],
  7: [],
  8: ['BITS_F412']},
 'A3': {3: ['MATH_F211', 'EEE_F211', 'EEE_F212', 'EEE_F215', 'EEE_F214'],
  4: ['ECON_F211',
   'MGTS_F211',
   'EEE_F241',
   'EEE_F243',
   'EEE_F244',
   'BITS_F225'],
  5: ['EEE_F311', 'MATH_F212', 'ME_F344', 'EEE_F313'],
  6: ['EEE_F341', 'EEE_F342', 'EEE_F312'],
  7: [],
  8: []},
 'AA': {3: ['MATH_F211', 'ECE_F211', 'ECE_F212', 'ECE_F215', 'ECE_F214'],
  4: ['ECON_F211',
   'MGTS_F211',
   'ECE

In [5]:
import json
with open("cdcdict.json", 'w') as f :
    json.dump(cdcs, f)